In [1]:
import cv2
import os

def extract_frames_from_videos(video_folder, output_folder, label):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        video_output_folder = os.path.join(output_folder, label, video_name)
        
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)
        
        # Check if frames are already extracted
        if len(os.listdir(video_output_folder)) > 0:
            print(f"Frames already extracted for {video_file}")
            continue
        
        cap = cv2.VideoCapture(video_path)
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_path = os.path.join(video_output_folder, f"{label}_frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_count += 1

        cap.release()
        print(f"Extracted {frame_count} frames from {video_file}")

# Extract frames for each class
extract_frames_from_videos("Dataset/Climb", "ExtractedFrames", "Climb")
extract_frames_from_videos("Dataset/Jump", "ExtractedFrames", "Jump")
extract_frames_from_videos("Dataset/Normal", "ExtractedFrames", "Normal")


Frames already extracted for 1.mp4
Frames already extracted for 10.mp4
Frames already extracted for 100.mp4
Frames already extracted for 101.mp4
Frames already extracted for 102.mp4
Frames already extracted for 103.mp4
Frames already extracted for 104.mp4
Frames already extracted for 105.mp4
Frames already extracted for 106.mp4
Frames already extracted for 107.mp4
Frames already extracted for 108.mp4
Frames already extracted for 109.mp4
Frames already extracted for 11.mp4
Frames already extracted for 110.mp4
Frames already extracted for 111.mp4
Frames already extracted for 112.mp4
Frames already extracted for 113.mp4
Frames already extracted for 114.mp4
Frames already extracted for 115.mp4
Frames already extracted for 116.mp4
Frames already extracted for 117.mp4
Frames already extracted for 118.mp4
Frames already extracted for 119.mp4
Frames already extracted for 12.mp4
Frames already extracted for 120.mp4
Frames already extracted for 13.mp4
Frames already extracted for 14.mp4
Frames a

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

def load_frames_from_folder(folder, img_size=(64, 64)):
    frames = []
    labels = []
    for class_folder in os.listdir(folder):
        class_folder_path = os.path.join(folder, class_folder)
        if os.path.isdir(class_folder_path):
            for video_folder in os.listdir(class_folder_path):
                video_folder_path = os.path.join(class_folder_path, video_folder)
                if os.path.isdir(video_folder_path):
                    for filename in os.listdir(video_folder_path):
                        if filename.endswith(".jpg"):
                            img_path = os.path.join(video_folder_path, filename)
                            try:
                                img = load_img(img_path, target_size=img_size)
                                img_array = img_to_array(img)
                                frames.append(img_array)
                                if 'Climb' in class_folder:
                                    labels.append(0)
                                elif 'Jump' in class_folder:
                                    labels.append(1)
                                elif 'Normal' in class_folder:
                                    labels.append(2)
                                else:
                                    print(f"Unknown class folder: {class_folder}")
                            except Exception as e:
                                print(f"Error loading image {img_path}: {e}")
    return np.array(frames), np.array(labels)

# Load frames and labels for all classes
frames, labels = load_frames_from_folder("ExtractedFrames")

# Print the number of frames loaded for each class
print(f"Total frames: {len(frames)}, Total labels: {len(labels)}")

# Combine and normalize
frames = frames / 255.0

# Print the number of combined frames and labels
print(f"Total frames: {len(frames)}, Total labels: {len(labels)}")


Total frames: 52206, Total labels: 52206
Total frames: 52206, Total labels: 52206


In [3]:
import numpy as np
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, frames, labels, batch_size=32, seq_length=20, img_size=(64, 64, 3), shuffle=True):
        self.frames = frames
        self.labels = labels
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.img_size = img_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.labels) - self.seq_length + 1)
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor((len(self.labels) - self.seq_length + 1) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, indexes):
        X = np.empty((self.batch_size, self.seq_length, *self.img_size))
        y = np.empty((self.batch_size), dtype=int)
        
        for i, idx in enumerate(indexes):
            X[i,] = self.frames[idx:idx + self.seq_length]
            y[i] = self.labels[idx + self.seq_length - 1]
        
        return X, y

# Create the data generators
training_generator = DataGenerator(frames, labels, batch_size=32)
validation_generator = DataGenerator(frames, labels, batch_size=32)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dense

def create_lrcn_model(input_shape):
    model = Sequential()
    
    # 2D CNN within TimeDistributed
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))
    
    # LSTM
    model.add(LSTM(100))
    
    # Dense Layer
    model.add(Dense(3, activation='softmax'))
    
    return model

# Define the input shape
input_shape = (20, 64, 64, 3)

# Create the model
model = create_lrcn_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 20, 62, 62, 64)   1792      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 31, 31, 64)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 20, 61504)        0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               24642000  
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                        

In [5]:
# Train the model using the generators
model.fit(training_generator, validation_data=validation_generator, epochs=5)

# Save the model
model.save('lrcn_model.h5')


Epoch 1/5
1630/1630 [==============================] - 2184s 1s/step - loss: 0.1170 - accuracy: 0.9588 - val_loss: 6.5303e-04 - val_accuracy: 1.0000
Epoch 2/5
1630/1630 [==============================] - 2200s 1s/step - loss: 3.2268e-04 - accuracy: 1.0000 - val_loss: 1.3738e-04 - val_accuracy: 1.0000
Epoch 3/5
1630/1630 [==============================] - 2179s 1s/step - loss: 0.0065 - accuracy: 0.9980 - val_loss: 1.7952e-04 - val_accuracy: 1.0000
Epoch 4/5
1630/1630 [==============================] - 2199s 1s/step - loss: 0.0199 - accuracy: 0.9951 - val_loss: 9.0988e-04 - val_accuracy: 1.0000
Epoch 5/5
1630/1630 [==============================] - 2223s 1s/step - loss: 3.5230e-04 - accuracy: 1.0000 - val_loss: 1.1243e-04 - val_accuracy: 1.0000


In [6]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('lrcn_model.h5')


In [10]:
import cv2
import numpy as np
import os

def video_to_frames(video_path, img_size=(64, 64)):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, img_size)
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Example usage
test_video_path = 'Test\Test2.mp4'
test_frames = video_to_frames(test_video_path)
print(f"Extracted {test_frames.shape[0]} frames from the test video.")


Extracted 634 frames from the test video.


In [11]:
def preprocess_frames(frames, seq_length=20):
    frames = frames / 255.0  # Normalize the frames
    sequences = []
    for i in range(len(frames) - seq_length + 1):
        sequences.append(frames[i:i + seq_length])
    return np.array(sequences)

# Example usage
seq_length = 20
test_sequences = preprocess_frames(test_frames, seq_length)
print(f"Created {test_sequences.shape[0]} sequences from the test frames.")


Created 615 sequences from the test frames.


In [12]:
def predict_actions(sequences, model):
    predictions = model.predict(sequences)
    predicted_classes = np.argmax(predictions, axis=1)
    actions = {0: 'Climb', 1: 'Jump', 2: 'Normal'}
    predicted_actions = [actions[pred] for pred in predicted_classes]
    return predicted_actions

# Example usage
predicted_actions = predict_actions(test_sequences, model)
for i, action in enumerate(predicted_actions):
    print(f"Sequence {i}: Predicted Action: {action}")


20/20 [==============================] - 5s 241ms/step
Sequence 0: Predicted Action: Normal
Sequence 1: Predicted Action: Normal
Sequence 2: Predicted Action: Normal
Sequence 3: Predicted Action: Normal
Sequence 4: Predicted Action: Normal
Sequence 5: Predicted Action: Normal
Sequence 6: Predicted Action: Normal
Sequence 7: Predicted Action: Normal
Sequence 8: Predicted Action: Normal
Sequence 9: Predicted Action: Normal
Sequence 10: Predicted Action: Normal
Sequence 11: Predicted Action: Normal
Sequence 12: Predicted Action: Normal
Sequence 13: Predicted Action: Normal
Sequence 14: Predicted Action: Normal
Sequence 15: Predicted Action: Normal
Sequence 16: Predicted Action: Normal
Sequence 17: Predicted Action: Normal
Sequence 18: Predicted Action: Normal
Sequence 19: Predicted Action: Normal
Sequence 20: Predicted Action: Normal
Sequence 21: Predicted Action: Normal
Sequence 22: Predicted Action: Normal
Sequence 23: Predicted Action: Normal
Sequence 24: Predicted Action: Normal
Seque